# Download the ZIP File from RunningAHEAD

The Export function is available only when logged into the RunningAHEAD website.

This block uses Selenium with the chromedriver. Ensure that the .env file contains the values for your username, password, ID, and name.

The file will download to the ~/Downloads folder as name.tab.zip.

In [1]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

driver.get("https://www.runningahead.com/")

driver.find_element(By.ID, "ctl00_ctl00_ctl00_SiteContent_PageContent_MainContent_email").send_keys(os.getenv("RA_USER"))
driver.find_element(By.ID, "ctl00_ctl00_ctl00_SiteContent_PageContent_MainContent_password").send_keys(os.getenv("RA_PASS"))
driver.find_element(By.ID, "ctl00_ctl00_ctl00_SiteContent_PageContent_MainContent_login_s").click()

driver.get("https://www.runningahead.com/logs/" + os.getenv("RA_ID") + "/tools/export")

driver.find_element(By.ID, "ctl00_ctl00_ctl00_SiteContent_PageContent_TrainingLogContent_Download_s").click()

time.sleep(10)

driver.quit()

In [2]:
import zipfile
import csv

expected_file = os.getenv("RA_FOLDER") + os.getenv("RA_NAME") + ".tab.zip"
with zipfile.ZipFile(expected_file, 'r') as zip_ref:
    zip_ref.extractall(os.getenv("RA_FOLDER") + "runningahead-logs/")

log_file = os.getenv("RA_FOLDER") + "runningahead-logs/log.txt"

with open(log_file, "r") as f:
    lines = f.readlines()

# The original file has an extra tab that messes up everything
new_header = "Date	TimeOfDay	Type	SubType	Distance	DistanceUnit	Duration	Weight	WeightUnit	RestHR	AvgHR	MaxHR	Sleep	Calories	Quality	Effort	Weather	Temperature	TempUnit	Notes	Course	CourseSurface	CourseNotes	ShoeMake	ShoeModel	Size	System	ShoeSerial	ShoePrice	OverallPlace	FieldSize	GroupMinAge	GroupMaxAge	GroupPlace	GroupSize	GenderPlace	GenderSize\n"

with open(log_file, "w") as f:
    f.write(new_header)

    count = 0
    for line in lines:
        if count > 0:
            f.write(line + "\n")
        count = count + 1

In [3]:
import duckdb

connection = duckdb.connect(database=':memory:')

log = connection.read_csv(log_file)

In [4]:
# Create a conversion table so everything can be converted through SQL
conversion = connection.sql("""
    SELECT 'Mile' AS Unit, 1 AS ConversionFactor
    UNION
    SELECT 'Kilometer' AS Unit, 0.6213712 AS ConversionFactor
    UNION
    SELECT 'Meter' AS Unit, 0.0006213712 AS ConversionFactor
""")

In [5]:
streak_log = connection.sql("""
    WITH convert_to_miles AS (
        SELECT
            l.Date
            , l.Distance * c.ConversionFactor AS DistanceMiles
        FROM log AS l
        INNER JOIN conversion AS c ON l.DistanceUnit = c.Unit
        WHERE Type = 'Run'
    ), aggregated_log AS ( 
        SELECT
            Date
            , SUM(DistanceMiles) AS Distance
        FROM convert_to_miles
        GROUP BY
            Date
    ), streak_groups AS (
        SELECT
            Date
            , Distance
            , CASE
                WHEN Date - 1 = LAG(Date) OVER (ORDER BY Date) AND Distance >= 1 THEN 0
                ELSE 1
            END AS StreakBreak
        FROM aggregated_log
    ), grouped_log AS (
        SELECT
            Date
            , Distance
            , SUM(StreakBreak) OVER (ORDER BY Date) AS StreakGroup
        FROM streak_groups
    )
    SELECT
        Date
        , Distance
        , StreakGroup
        , ROW_NUMBER() OVER (PARTITION BY StreakGroup ORDER BY Date) AS Streak
    FROM grouped_log
    ORDER BY Date;
""")

In [6]:
connection.sql("""
    SELECT
        StreakGroup
        , MIN(Date) AS min_day_of_streak
        , MAX(Date) AS max_day_of_streak
        , ROUND(SUM(Distance), 2) AS total_distance
        , MAX(Streak) AS total_days
    FROM streak_log
    GROUP BY
        StreakGroup
    HAVING total_days > 10
    ORDER BY total_days DESC, min_day_of_streak ASC
""")

┌─────────────┬───────────────────┬───────────────────┬────────────────┬────────────┐
│ StreakGroup │ min_day_of_streak │ max_day_of_streak │ total_distance │ total_days │
│   int128    │       date        │       date        │     double     │   int64    │
├─────────────┼───────────────────┼───────────────────┼────────────────┼────────────┤
│         134 │ 2011-08-31        │ 2012-04-26        │        1089.79 │        240 │
│         858 │ 2024-08-19        │ 2024-11-20        │          380.0 │         94 │
│         406 │ 2017-08-27        │ 2017-11-25        │         375.12 │         91 │
│         470 │ 2018-11-12        │ 2019-02-08        │         338.14 │         89 │
│         249 │ 2013-09-21        │ 2013-10-09        │          97.47 │         19 │
│         596 │ 2020-10-18        │ 2020-11-05        │         135.17 │         19 │
│         857 │ 2024-07-29        │ 2024-08-14        │          55.78 │         17 │
│         294 │ 2014-06-29        │ 2014-07-14        

In [7]:
import shutil

# clean up the ZIP file
os.remove(expected_file)

# clean up the extracted files
files = os.getenv("RA_FOLDER") + "runningahead-logs/"
shutil.rmtree(files)